In [15]:
import xarray as xr
import os
import pystac
import pystac_client
import stackstac
from dask.distributed import Client
import dask
import json

In [2]:
# GDAL environment variables for better performance
os.environ['AWS_REGION']='us-west-2'
os.environ['GDAL_DISABLE_READDIR_ON_OPEN']='EMPTY_DIR' 
os.environ['AWS_NO_SIGN_REQUEST']='YES' 

In [3]:
bbox = {
    "type": "Polygon",
    "coordinates": [
          [
            [75.42382800808971,36.41082887114753],
            [75.19442677164156,36.41082887114753],
            [75.19442677164156,36.201076360872946],
            [75.42382800808971,36.201076360872946],
            [75.42382800808971,36.41082887114753]
          ]
        ],
}

In [4]:
# We use the api from element84 to query the data
URL = "https://earth-search.aws.element84.com/v1"
catalog = pystac_client.Client.open(URL)

In [5]:
search = catalog.search(
    collections=["sentinel-2-l2a"],
    intersects=bbox,
    query={"eo:cloud_cover": {"lt": 10}}, # less than 10% cloud cover
)

# Check how many items were returned
items = search.item_collection()
print(f"Returned {len(items)} Items")

Returned 182 Items


In [6]:
sentinel2_stack = stackstac.stack(items)

/srv/conda/envs/image-correlation/lib/python3.12/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


In [7]:
# select images from May-September to minimize snow cover
sentinel2_stack_snowoff = sentinel2_stack.where((sentinel2_stack.time.dt.month > 5) & (sentinel2_stack.time.dt.month < 10), drop=True)

In [19]:
print(image_dates)

['2018-06-02', '2018-06-07', '2018-07-15', '2018-08-04', '2018-09-18', '2019-07-05', '2019-09-28', '2020-06-29', '2020-07-24', '2020-07-29', '2020-08-23', '2020-08-25', '2020-08-25', '2020-08-25', '2020-08-25', '2020-09-22', '2021-06-06', '2021-06-09', '2021-09-02', '2021-09-04', '2021-09-07', '2021-09-07', '2021-09-09', '2021-09-09', '2021-09-19', '2021-09-27', '2021-09-29', '2022-06-24', '2022-07-01', '2022-07-04', '2022-07-19', '2022-07-21', '2022-09-19', '2022-09-27', '2022-09-29', '2023-06-19', '2023-06-21', '2023-06-24', '2023-07-14', '2023-08-13', '2023-08-30', '2023-09-12']


In [20]:
image_dates = sentinel2_stack_snowoff.time.dt.strftime('%Y-%m-%d').values.tolist()

In [21]:
npairs = 3

In [22]:
pairs = []
for r in range(len(image_dates) - npairs):
    for s in range(1, npairs + 1 ):
      img1_date = image_dates[r]
      img2_date = image_dates[r+s]
      shortname = f'{img1_date}_{img2_date}'
      pairs.append({'img1_date': img1_date, 'img2_date': img2_date, 'name':shortname})
matrixJSON = f'{{"include":{json.dumps(pairs)}}}'
print(f'Number of Interferograms: {len(pairs)}')

with open(os.environ['GITHUB_OUTPUT'], 'a') as f:
    print(f'BURST_IDS={image_dates}', file=f)
    print(f'MATRIX_PARAMS_COMBINATIONS={matrixJSON}', file=f)

Number of Interferograms: 117


KeyError: 'GITHUB_OUTPUT'